# Step 0.1. Import necessary libraries 

In [ ]:
# Standard python libraries
import logging
import os
import time

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, median_absolute_error
from sklearn.model_selection import train_test_split
import torch
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

from lightautoml.tasks import Task
from lightautoml.addons.autots.base import AutoTS

In [ ]:
def dict_to_str(d):
    s = ''
    for key, value in d.items():
        s += f'_{key}:{value}'
    return s

In [ ]:
PATH = '../../LAMA/LightAutoML/ts_example_anon/all_anonimized'

STEP = 7
PLOT = True

seq_params = {'seq0': {'case': 'next_values', 
                           'params': {'n_target': STEP,
                                      'history': np.maximum(7, STEP),
                                      'step': 1, 'from_last': True}},}

trend_params_0 = {'trend': True, 
                  'train_on_trend': False,
                  'trend_type': 'decompose', # 'decompose', 'decompose_STL', 'linear' or 'rolling'
                  'trend_size': 7, 
                  'decompose_period': 30, 
                  'detect_step_quantile': 0.01, 
                  'detect_step_window': 7, 
                  'detect_step_threshold': 0.7,
                  'rolling_size': 7, 
                  'verbose': 0}

In [ ]:
for trend_params in [trend_params_0]:
    NAME = dict_to_str(trend_params)
    i = 0
    for seq in tqdm(os.listdir(PATH)):
        cur_path = os.path.join(PATH, seq)
        df = pd.read_csv(cur_path)
        df = df.rename(columns={'Unnamed: 0': 'date', seq.split('.')[0]: 'signal'})

        train = df.iloc[:-STEP].copy()
        test = df.iloc[-STEP:].copy()

        roles = {'target': 'signal'}
        task = Task('multi:reg', greater_is_better=False, metric='mae', loss='mae')
        model = AutoTS(task, seq_params=seq_params, trend_params=trend_params)

        oof_pred_seq, median = model.fit_predict(train, roles)
        pred, trend = model.predict(train)
        print(f'MAE: {np.round(median_absolute_error(test.signal.values[-STEP:], pred), 4)}')
        
        if PLOT:
            plt.figure(figsize=[10, 5])
            plt.plot(train.signal.values.tolist() + [np.NaN]*len(test), c='b', label='Train')
            plt.plot(median.tolist() + [np.NaN]*len(test), c='k', label='trend oof')
            plt.plot([np.NaN]*len(train) + test.signal.values[-STEP:].tolist(), c='g', label='True')
            plt.plot([np.NaN]*len(train) + trend.tolist(), c='k',label='Trend')
            plt.plot([np.NaN]*len(train) + pred.tolist(), c='r', label='Pred')
            plt.title(f'{seq} MAE: {np.round(median_absolute_error(test.signal.values[-STEP:], pred), 4)}')
            plt.legend()
            plt.show()
        print('=============')



        i+=1